In [21]:
import pandas as pd
import re
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras import layers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [57]:
train = pd.read_csv('csv/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [56]:
test = pd.read_csv('csv/test.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


# BERT

In [66]:
len_embedding = 768

In [67]:
normalized_embeddings_train = np.loadtxt("train_embedding_file_bert.txt").reshape(len(train.index),len_embedding)
normalized_embeddings_train.shape

(7613, 768)

In [68]:
normalized_embeddings_test = np.loadtxt("test_embedding_file_bert.txt").reshape(len(test.index),len_embedding)
normalized_embeddings_test.shape

(3263, 768)

In [69]:
X, y = normalized_embeddings_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [70]:
model = Sequential()
model.add(layers.Dense(512, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(256, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [71]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 512)               393728    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 257       
Total params: 525,313
Trainable params: 525,313
Non-trainable params: 0
_________________________________________________________________


In [73]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 7s 10ms/step - loss: 0.5378 - accuracy: 0.7415 - val_loss: 0.4481 - val_accuracy: 0.8189
Epoch 2/10
686/686 [==============================] - 7s 10ms/step - loss: 0.4709 - accuracy: 0.7870 - val_loss: 0.4269 - val_accuracy: 0.8163
Epoch 3/10
686/686 [==============================] - 7s 10ms/step - loss: 0.4503 - accuracy: 0.8019 - val_loss: 0.4172 - val_accuracy: 0.8110
Epoch 4/10
686/686 [==============================] - 7s 10ms/step - loss: 0.4310 - accuracy: 0.8078 - val_loss: 0.4135 - val_accuracy: 0.8176
Epoch 5/10
686/686 [==============================] - 7s 10ms/step - loss: 0.4170 - accuracy: 0.8156 - val_loss: 0.4152 - val_accuracy: 0.8189
Epoch 6/10
686/686 [==============================] - 7s 10ms/step - loss: 0.4061 - accuracy: 0.8183 - val_loss: 0.4266 - val_accuracy: 0.8150
Epoch 7/10
686/686 [==============================] - 6s 9ms/step - loss: 0.3971 - accuracy: 0.8247 - val_loss: 0.4091 - val_accuracy: 0.8241


In [75]:
test['target'] = test_pred_int
test.drop(columns=['keyword','location','text'],inplace=True)
test.reset_index(inplace=True)
test.head()

,target
id,
0,1
2,1
3,1
9,1
11,1


In [84]:
test.to_csv('csv/test_red_bert.csv')

In [85]:
train_pred = model.predict(normalized_embeddings_train)
train_pred_int = train_pred.round().astype('int')

In [86]:
train['bert_net'] = train_pred_int
train = train['bert_net'].to_frame()
train.to_csv('csv/train_red_bert.csv')

# ELMo

In [87]:
len_embedding = 1024

In [88]:
normalized_embeddings_train = np.loadtxt("train_embedding_file.txt").reshape(len(train.index),len_embedding)
normalized_embeddings_train.shape

(7613, 1024)

In [89]:
normalized_embeddings_test = np.loadtxt("test_embedding_file.txt").reshape(len(test.index),len_embedding)
normalized_embeddings_test.shape

(3263, 1024)

In [90]:
train = pd.read_csv('csv/train.csv')
test = pd.read_csv('csv/test.csv')
X, y = normalized_embeddings_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [91]:
model = Sequential()
model.add(layers.Dense(512, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(256, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [92]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 257       
Total params: 656,385
Trainable params: 656,385
Non-trainable params: 0
_________________________________________________________________


In [93]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 7s 10ms/step - loss: 0.5334 - accuracy: 0.7565 - val_loss: 0.4555 - val_accuracy: 0.7992
Epoch 2/10
686/686 [==============================] - 6s 9ms/step - loss: 0.4532 - accuracy: 0.7990 - val_loss: 0.4394 - val_accuracy: 0.8150
Epoch 3/10
686/686 [==============================] - 7s 10ms/step - loss: 0.4355 - accuracy: 0.8041 - val_loss: 0.4365 - val_accuracy: 0.8031
Epoch 4/10
686/686 [==============================] - 7s 10ms/step - loss: 0.4217 - accuracy: 0.8101 - val_loss: 0.4289 - val_accuracy: 0.8150
Epoch 5/10
686/686 [==============================] - 7s 10ms/step - loss: 0.4057 - accuracy: 0.8257 - val_loss: 0.4255 - val_accuracy: 0.8136
Epoch 6/10
686/686 [==============================] - 7s 10ms/step - loss: 0.3964 - accuracy: 0.8264 - val_loss: 0.4237 - val_accuracy: 0.8189
Epoch 7/10
686/686 [==============================] - 7s 10ms/step - loss: 0.3833 - accuracy: 0.8318 - val_loss: 0.4265 - val_accuracy: 0.8228


In [94]:
test_pred = model.predict(normalized_embeddings_test)
test_pred_int = test_pred.round().astype('int')

In [95]:
test['elmo_net'] = test_pred_int
test = test['elmo_net'].to_frame()
test.head()

,elmo_net
0,1
1,1
2,1
3,1
4,1


In [96]:
test.to_csv('csv/test_red_elmo.csv')

In [97]:
train_pred = model.predict(normalized_embeddings_train)
train_pred_int = train_pred.round().astype('int')

In [98]:
train['elmo_net'] = train_pred_int
train = train['elmo_net'].to_frame()
train.to_csv('csv/train_red_elmo.csv')

# W2V

In [99]:
len_embedding = 300

In [100]:
normalized_embeddings_train = np.loadtxt("train_embedding_file_w2v(norm).txt").reshape(len(train.index),len_embedding)
normalized_embeddings_train.shape

(7613, 300)

In [101]:
normalized_embeddings_test = np.loadtxt("test_embedding_file_w2v(norm).txt").reshape(len(test.index),len_embedding)
normalized_embeddings_test.shape

(3263, 300)

In [102]:
train = pd.read_csv('csv/train.csv')
test = pd.read_csv('csv/test.csv')
X, y = normalized_embeddings_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [103]:
model = Sequential()
model.add(layers.Dense(200, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(100, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [104]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 200)               60200     
_________________________________________________________________
dropout_12 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 101       
Total params: 80,401
Trainable params: 80,401
Non-trainable params: 0
_________________________________________________________________


In [105]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 2s 4ms/step - loss: 0.6151 - accuracy: 0.6818 - val_loss: 0.5145 - val_accuracy: 0.7835
Epoch 2/10
686/686 [==============================] - 2s 3ms/step - loss: 0.5123 - accuracy: 0.7625 - val_loss: 0.4637 - val_accuracy: 0.7940
Epoch 3/10
686/686 [==============================] - 2s 3ms/step - loss: 0.4791 - accuracy: 0.7837 - val_loss: 0.4483 - val_accuracy: 0.8058
Epoch 4/10
686/686 [==============================] - 2s 4ms/step - loss: 0.4668 - accuracy: 0.7867 - val_loss: 0.4392 - val_accuracy: 0.8136
Epoch 5/10
686/686 [==============================] - 2s 3ms/step - loss: 0.4535 - accuracy: 0.7946 - val_loss: 0.4334 - val_accuracy: 0.8097
Epoch 6/10
686/686 [==============================] - 2s 2ms/step - loss: 0.4474 - accuracy: 0.8019 - val_loss: 0.4286 - val_accuracy: 0.8136
Epoch 7/10
686/686 [==============================] - 2s 3ms/step - loss: 0.4343 - accuracy: 0.8095 - val_loss: 0.4249 - val_accuracy: 0.8150
Epoch 

In [106]:
test_pred = model.predict(normalized_embeddings_test)
test_pred_int = test_pred.round().astype('int')

In [107]:
test['w2v_net'] = test_pred_int
test = test['w2v_net'].to_frame()
test.head()

,w2v_net
0,1
1,1
2,1
3,1
4,1


In [108]:
test.to_csv('csv/test_red_w2v.csv')

In [109]:
train_pred = model.predict(normalized_embeddings_train)
train_pred_int = train_pred.round().astype('int')

In [110]:
train['w2v_net'] = train_pred_int
train = train['w2v_net'].to_frame()
train.to_csv('csv/train_red_w2v.csv')

# GLOve

In [112]:
len_embedding = 200

In [113]:
normalized_embeddings_train = np.loadtxt("train_embedding_file_glove.txt").reshape(len(train.index),len_embedding)
normalized_embeddings_train.shape

(7613, 200)

In [114]:
normalized_embeddings_test = np.loadtxt("test_embedding_file_glove.txt").reshape(len(test.index),len_embedding)
normalized_embeddings_test.shape

(3263, 200)

In [115]:
train = pd.read_csv('csv/train.csv')
test = pd.read_csv('csv/test.csv')
X, y = normalized_embeddings_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [116]:
model = Sequential()
model.add(layers.Dense(100, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(50, input_dim=len_embedding, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

In [117]:
optimizer=Adam(learning_rate=5e-5)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_14 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_15 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 51        
Total params: 25,201
Trainable params: 25,201
Non-trainable params: 0
_________________________________________________________________


In [118]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
686/686 [==============================] - 2s 2ms/step - loss: 0.9222 - accuracy: 0.6009 - val_loss: 0.5341 - val_accuracy: 0.7782
Epoch 2/10
686/686 [==============================] - 2s 2ms/step - loss: 0.7231 - accuracy: 0.6730 - val_loss: 0.5073 - val_accuracy: 0.8005
Epoch 3/10
686/686 [==============================] - 2s 2ms/step - loss: 0.6186 - accuracy: 0.7087 - val_loss: 0.4753 - val_accuracy: 0.8018
Epoch 4/10
686/686 [==============================] - 2s 2ms/step - loss: 0.5829 - accuracy: 0.7262 - val_loss: 0.4624 - val_accuracy: 0.8097
Epoch 5/10
686/686 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7453 - val_loss: 0.4526 - val_accuracy: 0.8110
Epoch 6/10
686/686 [==============================] - 2s 2ms/step - loss: 0.5268 - accuracy: 0.7577 - val_loss: 0.4433 - val_accuracy: 0.8189
Epoch 7/10
686/686 [==============================] - 2s 2ms/step - loss: 0.5162 - accuracy: 0.7651 - val_loss: 0.4399 - val_accuracy: 0.8150
Epoch 

In [119]:
test_pred = model.predict(normalized_embeddings_test)
test_pred_int = test_pred.round().astype('int')

In [120]:
test['glove_net'] = test_pred_int
test = test['glove_net'].to_frame()
test.head()

,glove_net
0,1
1,1
2,1
3,1
4,1


In [121]:
test.to_csv('csv/test_red_glove.csv')

In [122]:
train_pred = model.predict(normalized_embeddings_train)
train_pred_int = train_pred.round().astype('int')

In [123]:
train['glove_net'] = train_pred_int
train = train['glove_net'].to_frame()
train.to_csv('csv/train_red_glove.csv')